In [1]:
# Fixed and tested-looking Pareto pipeline (syntactically clean)
# Save as pareto_pipeline_fixed.py and run in an environment with xpress, numpy, pandas, matplotlib

import xpress as xp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import os
from typing import List, Dict, Optional

# --------------------------- USER CONFIG ---------------------------
POIS_PATH = 'reduced_pois2.csv'
STATIONS_TEMPLATE = 'candidate_stations_P300_type_{}.csv'
DEMAND_TEMPLATE = 'poi_demand_per_hour_type_{}_cluster{}.csv'
EXISTING_STATION_DATA = 'station_data.csv'

STRATEGIES = ['uniform', 'commercial', 'pred']

INITIAL_BUDGET = 5_000_000
MIN_BUDGET = 100_000
BUDGET_STEP = 300_000

Tau = 2
k = 1000

COST_EXISTING = 60000
COST_NEW = 90000
ALPHA = 0.7

W_C = {
    'library': 1,
    'residential': 1,
    'school': 1,
    'commercial': 1,
    'university': 1,
    'hospital': 1
}

XPRESS_VERBOSE = False
OUTPUT_DIR = 'pareto_outputs'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --------------------------- UTILITIES ---------------------------

def haversine_matrix(pois: pd.DataFrame,
                     stations: pd.DataFrame,
                     poi_lat: str = 'lat', poi_lon: str = 'lon',
                     st_lat: str = 'centroid_lat', st_lon: str = 'centroid_lon') -> np.ndarray:
    """Compute POI x Station distance matrix (km)."""
    poi_coords = pois[[poi_lat, poi_lon]].to_numpy()
    st_coords = stations[[st_lat, st_lon]].to_numpy()
    lat1 = np.radians(poi_coords[:, 0])[:, None]
    lon1 = np.radians(poi_coords[:, 1])[:, None]
    lat2 = np.radians(st_coords[:, 0])[None, :]
    lon2 = np.radians(st_coords[:, 1])[None, :]
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    R = 6371.0
    return R * c

# --------------------------- MODEL BUILDER ---------------------------

def build_and_solve_model(distance_matrix_km: np.ndarray,
                          stations: pd.DataFrame,
                          pois: pd.DataFrame,
                          demand_scenarios: List[List[float]],
                          capacity_vector: np.ndarray,
                          cost_vector: np.ndarray,
                          I_c: Dict[str, List[int]],
                          w_c: Dict[str, float],
                          Tau: float,
                          k: float,
                          budget_limit: float,
                          verbose: bool = False) -> Optional[Dict]:
    I, J = distance_matrix_km.shape
    prob = xp.problem(name='pcentre_modular_run')

    # Variables
    Y = {j: prob.addVariable(vartype=xp.binary, name=f'Y_{j}') for j in range(J)}
    X = {(i, j): prob.addVariable(vartype=xp.binary, name=f'X_{i}_{j}') for i in range(I) for j in range(J)}
    Z = {j: prob.addVariable(vartype=xp.integer, name=f'Z_{j}') for j in range(J)}
    Q = prob.addVariable(name='Q', lb=0)

    # Constraints
    for i in range(I):
        prob.addConstraint(xp.Sum(X[i, j] for j in range(J)) <= 1)

    for i in range(I):
        for j in range(J):
            prob.addConstraint(X[i, j] <= Y[j])

    for i in range(I):
        prob.addConstraint(xp.Sum(distance_matrix_km[i, j] * X[i, j] for j in range(J)) <= Q)

    prob.addConstraint(
        xp.Sum(cost_vector[j] * Y[j] for j in range(J))
        + k * xp.Sum((Z[j] - capacity_vector[j] * Y[j]) for j in range(J))
        <= budget_limit
    )

    for j in range(J):
        prob.addConstraint(Z[j] >= capacity_vector[j] * Y[j])
        prob.addConstraint(Z[j] <= 100 * Y[j])
        for s in range(len(demand_scenarios)):
            ds = demand_scenarios[s]
            prob.addConstraint(xp.Sum(X[i, j] * ds[i] for i in range(I)) <= Tau * Z[j])

    for s in range(len(demand_scenarios)):
        ds = demand_scenarios[s]
        # Option B: Binary hit-based service level (% of POIs in category served)
        for c, idx_list in I_c.items():
            if not idx_list:
                continue
            lhs = xp.Sum(xp.Sum(X[i, j] for j in range(J)) for i in idx_list)
            rhs = w_c[c] * len(idx_list)
            prob.addConstraint(lhs >= rhs)

    prob.setObjective(Q, sense=xp.minimize)

    xp.setOutputEnabled(verbose)
    try:
        prob.solve()
    except Exception as e:
        if verbose:
            print('Solver failed:', e)
        return None

    try:
        Q_val = prob.getSolution(Q)
    except Exception:
        return None

    Y_sol = np.array([prob.getSolution(Y[j]) for j in range(J)])
    Z_sol = np.array([prob.getSolution(Z[j]) for j in range(J)])

    budget_used = float(np.dot(cost_vector, Y_sol) + k * np.sum(Z_sol - capacity_vector * Y_sol))
    opened_count = int((Y_sol > 0.5).sum())

    opened_indices = [int(j) for j in range(J) if Y_sol[j] > 0.5]
    Z_values_opened = {int(j): int(Z_sol[j]) for j in opened_indices}

    X_sol = {(i, j): prob.getSolution(X[(i, j)]) for i in range(I) for j in range(J)}

    # Build solution matrix
    solution_matrix = np.zeros((I, J))
    for (i, j), val in X_sol.items():
        solution_matrix[i, j] = val

    # Assign each POI to the station with max X_ij
    assignments = np.argmax(solution_matrix, axis=1)

    return {'problem': prob,
            'Q': Q_val,
            'Y_sol': Y_sol,
            'Z_sol': Z_sol,
            'X_sol': X_sol,
            'solution_matrix': solution_matrix,
            'assignments': assignments,
            'budget_used': budget_used,
            'opened': opened_count,
            'opened_indices': opened_indices,
            'Z_values_opened': Z_values_opened}

# --------------------------- PARETO FOR STRATEGY (logged + store) ---------------------------

def pareto_for_strategy_store(strategy: str,
                              pois_path: str,
                              stations_template: str,
                              demand_template: str,
                              existing_station_data_path: str,
                              initial_budget: int = INITIAL_BUDGET,
                              min_budget: int = MIN_BUDGET,
                              budget_step: int = BUDGET_STEP,
                              Tau: float = Tau,
                              k: float = k,
                              w_c: Dict[str, float] = W_C,
                              alpha: float = ALPHA,
                              cost_existing: float = COST_EXISTING,
                              cost_new: float = COST_NEW,
                              verbose: bool = XPRESS_VERBOSE) -> Optional[pd.DataFrame]:
    pois = pd.read_csv(pois_path)
    stations = pd.read_csv(stations_template.format(strategy))
    D_km = haversine_matrix(pois, stations)

    # nearest-POI category assignment
    poi_categories = pois['category'].reset_index(drop=True)
    min_row_idx_for_station = np.argmin(D_km, axis=0)
    station_categories = poi_categories.iloc[min_row_idx_for_station].reset_index(drop=True)
    stations['category'] = station_categories.values

    categories = list(w_c.keys())
    I_c = {c: pois.index[pois['category'] == c].tolist() for c in categories}

    demand_scenarios: List[List[float]] = []
    for i in range(4):
        df = pd.read_csv(demand_template.format(strategy, i))
        demand_scenarios.append(df['demand_per_hour'].tolist())

    existing_data = pd.read_csv(existing_station_data_path)
    capacity_map = dict(zip(existing_data['station_id'], existing_data['capacity']))
    capacity_vector = np.array([capacity_map.get(sid, 10) for sid in stations['snapped_station_id']])

    is_existing = stations['is_existing_station'].astype(bool).values
    station_is_commercial_by_nearest = (stations['category'] == 'commercial').values

    J = stations.shape[0]
    cost_vector = np.zeros(J, dtype=float)
    for j in range(J):
        if is_existing[j] and station_is_commercial_by_nearest[j]:
            cost_vector[j] = alpha * cost_existing
        elif is_existing[j]:
            cost_vector[j] = cost_existing
        elif station_is_commercial_by_nearest[j]:
            cost_vector[j] = alpha * cost_new
        else:
            cost_vector[j] = cost_new

    # Budgets descending (high -> low)
    budgets = list(range(int(initial_budget), int(min_budget) - 1, -int(budget_step)))
    results = []
    detailed_solutions = []

    for b in budgets:
        print(f"[Strategy={strategy}] Attempting budget limit = £{b:,}")
        sol = build_and_solve_model(D_km, stations, pois, demand_scenarios,
                                    capacity_vector, cost_vector, I_c, w_c,
                                    Tau, k, float(b), verbose=verbose)
        if sol is None:
            print(f"[Strategy={strategy}] No feasible solution at budget £{b:,}")
            continue

        print(f"[Strategy={strategy}] Solved: budget_limit=£{b:,} -> budget_used=£{sol['budget_used']:.2f}, Q={sol['Q']:.4f} km, opened={sol['opened']} stations")

        results.append({'strategy': strategy, 'budget_limit': b, 'budget_used': sol['budget_used'], 'Q': sol['Q'], 'opened': sol['opened']})

        detailed_solutions.append({
            'strategy': strategy,
            'budget_limit': int(b),
            'budget_used': float(sol['budget_used']),
            'Q': float(sol['Q']),
            'opened_count': int(sol['opened']),
            'opened_indices': sol['opened_indices'],
            'Z_values_opened': sol['Z_values_opened']
        })

    # Save detailed solutions
    if detailed_solutions:
        det_csv = os.path.join(OUTPUT_DIR, f'detailed_solutions_{strategy}.csv')
        det_pkl = os.path.join(OUTPUT_DIR, f'detailed_solutions_{strategy}.pkl')
        pd.DataFrame(detailed_solutions).to_csv(det_csv, index=False)
        with open(det_pkl, 'wb') as f:
            pickle.dump(detailed_solutions, f)
        print(f"Saved detailed solutions for strategy {strategy} to {det_csv} and {det_pkl}")

    if len(results) == 0:
        return None

    df = pd.DataFrame(results).sort_values('budget_used', ascending=True).reset_index(drop=True)
    df.to_csv(os.path.join(OUTPUT_DIR, f'pareto_points_{strategy}.csv'), index=False)
    print(f"Saved pareto points for strategy {strategy} to {os.path.join(OUTPUT_DIR, f'pareto_points_{strategy}.csv')}")

    return df

# --------------------------- DOMINATION CHECK ---------------------------

def find_nondominated(df_all: pd.DataFrame) -> pd.DataFrame:
    pts = df_all[['budget_used', 'Q']].to_numpy()
    n = pts.shape[0]
    is_nd = np.ones(n, dtype=bool)
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            if (pts[j, 0] <= pts[i, 0] and pts[j, 1] <= pts[i, 1]) and (pts[j, 0] < pts[i, 0] or pts[j, 1] < pts[i, 1]):
                is_nd[i] = False
                break
    return df_all[is_nd].reset_index(drop=True)

# --------------------------- MAIN: RUN ALL + PLOT + STORE DOMINANT ---------------------------

def run_all_and_store_dominant(strategies: List[str] = STRATEGIES):
    per_strategy_dfs: Dict[str, pd.DataFrame] = {}

    for s in strategies:
        print('\n' + '=' * 70)
        print(f"Starting strategy: {s}")
        df = pareto_for_strategy_store(s, POIS_PATH, STATIONS_TEMPLATE, DEMAND_TEMPLATE, EXISTING_STATION_DATA,
                                      initial_budget=INITIAL_BUDGET, min_budget=MIN_BUDGET, budget_step=BUDGET_STEP,
                                      Tau=Tau, k=k, w_c=W_C, alpha=ALPHA, cost_existing=COST_EXISTING,
                                      cost_new=COST_NEW, verbose=XPRESS_VERBOSE)
        if df is None or df.empty:
            print(f"Strategy {s} produced no feasible points.")
            continue
        per_strategy_dfs[s] = df

    # Save per-strategy files already saved in function; combine
    all_dfs: List[pd.DataFrame] = []
    for s, df in per_strategy_dfs.items():
        tmp = df.copy()
        tmp['strategy'] = s
        all_dfs.append(tmp)

    if not all_dfs:
        print('No solutions found across strategies.')
        return

    df_all = pd.concat(all_dfs, ignore_index=True)
    df_all_path = os.path.join(OUTPUT_DIR, 'pareto_points_all_strategies.csv')
    df_all.to_csv(df_all_path, index=False)
    print(f'Saved all pareto points to {df_all_path}')

    # Find non-dominated
    df_nd = find_nondominated(df_all)
    nd_path = os.path.join(OUTPUT_DIR, 'pareto_points_nondominated.csv')
    df_nd.to_csv(nd_path, index=False)
    print(f'Saved nondominated (global Pareto) points to {nd_path}')

    # Save detailed dominant solutions from per-strategy pickles
    dominant_details = []
    for _, row in df_nd.iterrows():
        strat = row['strategy']
        pkl_path = os.path.join(OUTPUT_DIR, f'detailed_solutions_{strat}.pkl')
        if not os.path.exists(pkl_path):
            continue
        with open(pkl_path, 'rb') as f:
            det_list = pickle.load(f)
        # find entry with closest budget_used
        best = None
        min_diff = float('inf')
        for det in det_list:
            diff = abs(det['budget_used'] - row['budget_used'])
            if diff < min_diff:
                min_diff = diff
                best = det
        if best is not None:
            best_copy = best.copy()
            best_copy['strategy'] = strat
            dominant_details.append(best_copy)

    dominant_csv = os.path.join(OUTPUT_DIR, 'dominant_solutions_detailed.csv')
    dominant_pkl = os.path.join(OUTPUT_DIR, 'dominant_solutions_detailed.pkl')
    if dominant_details:
        pd.DataFrame(dominant_details).to_csv(dominant_csv, index=False)
        with open(dominant_pkl, 'wb') as f:
            pickle.dump(dominant_details, f)
        print(f'Saved dominant solution details to {dominant_csv} and {dominant_pkl}')
    else:
        print('No detailed dominant solutions found to save')

    # Side-by-side plot
    n = len(strategies)
    fig, axes = plt.subplots(1, n, figsize=(6 * n, 5), sharey=True)
    if n == 1:
        axes = [axes]
    for ax, s in zip(axes, strategies):
        df = per_strategy_dfs.get(s)
        if df is None or df.empty:
            ax.set_title(s + ' (no points)')
            continue
        ax.scatter(df['budget_used'], df['Q'], marker='o')
        for _, row in df.iterrows():
            ax.annotate(f"Q={row['Q']:.3f}\n£{row['budget_used']:.0f}\nopen={row['opened']}",
                        (row['budget_used'], row['Q']), textcoords='offset points', xytext=(3, 3), fontsize=8)
        ax.set_title(s)
        ax.set_xlabel('Budget used (£)')
        ax.grid(True)
    axes[0].set_ylabel('Q (km)')
    plt.suptitle('Per-strategy Pareto points (Budget used vs Q)')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    side_by_side_path = os.path.join(OUTPUT_DIR, 'pareto_per_strategy_side_by_side.png')
    plt.savefig(side_by_side_path, dpi=200)
    print(f'Saved side-by-side figure: {side_by_side_path}')

    # Combined plot with nondominated highlighted
    plt.figure(figsize=(9, 6))
    for s in strategies:
        sub = df_all[df_all['strategy'] == s]
        if sub.empty:
            continue
        plt.scatter(sub['budget_used'], sub['Q'], label=s, alpha=0.4)
    if not df_nd.empty:
        plt.scatter(df_nd['budget_used'], df_nd['Q'], color='red', s=100, label='Global Pareto (nondominated)')
        for _, row in df_nd.iterrows():
            plt.annotate(f"{row['strategy']}\nQ={row['Q']:.3f}\n£{row['budget_used']:.0f}",
                         (row['budget_used'], row['Q']), textcoords='offset points', xytext=(5, -5), fontsize=9)
    plt.xlabel('Budget used (£)')
    plt.ylabel('Q (km)')
    plt.title('Combined solutions and global Pareto (nondominated)')
    plt.legend()
    plt.grid(True)
    combined_path = os.path.join(OUTPUT_DIR, 'pareto_combined_nondominated.png')
    plt.tight_layout()
    plt.savefig(combined_path, dpi=200)
    print(f'Saved combined Pareto figure with nondominated highlighted: {combined_path}')

    print('\nGLOBAL NONDOMINATED SOLUTIONS:')
    print(df_nd)
    print('\nDetailed dominant solutions saved (if any) to:')
    print(dominant_csv)
    print(dominant_pkl)


In [2]:
"""
alpha_sensitivity_vs_uniform.py

Find values of alpha (cost scaling factor) at which commercial and pred strategies
become more worthwhile than uniform, under a fixed budget.

Comparison rule (Option C - dominance): strategy S is "preferred" over uniform if
both
  Q_S <= Q_uniform and budget_used_S <= budget_used_uniform
and at least one of those inequalities is strict.

Outputs:
 - CSV with alpha, Q and budget for each strategy and boolean flags indicating dominance
 - Plot Q vs alpha for the three strategies with markers where dominance occurs

Usage: python alpha_sensitivity_vs_uniform.py
"""

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Dict

OUTPUT_DIR = 'pareto_outputs'

os.makedirs(OUTPUT_DIR, exist_ok=True)

# -------------------- User settings --------------------
strategy_list = ['uniform', 'commercial', 'pred']
fixed_budget = 6_750_000  # change if you want a different budget
alpha_grid = np.linspace(0.01, 1.0, 15)  # grid of alpha to test

# -------------------- Data loader (copied logic from pipeline) --------------------

def load_data_for_strategy(strategy: str):
    pois = pd.read_csv(POIS_PATH)
    stations = pd.read_csv(STATIONS_TEMPLATE.format(strategy))
    D_km = haversine_matrix(pois, stations)

    poi_categories = pois['category'].reset_index(drop=True)
    min_row_idx_for_station = np.argmin(D_km, axis=0)
    station_categories = poi_categories.iloc[min_row_idx_for_station].reset_index(drop=True)
    stations['category'] = station_categories.values

    categories = list(W_C.keys())
    I_c = {c: pois.index[pois['category'] == c].tolist() for c in categories}

    demand_scenarios: List[List[float]] = []
    for i in range(4):
        df = pd.read_csv(DEMAND_TEMPLATE.format(strategy, i))
        demand_scenarios.append(df['demand_per_hour'].tolist())

    existing_data = pd.read_csv(EXISTING_STATION_DATA)
    capacity_map = dict(zip(existing_data['station_id'], existing_data['capacity']))
    capacity_vector = np.array([capacity_map.get(sid, 10) for sid in stations['snapped_station_id']])

    is_existing = stations['is_existing_station'].astype(bool).values
    station_is_commercial_by_nearest = (stations['category'] == 'commercial').values

    return {
        'pois': pois,
        'stations': stations,
        'D_km': D_km,
        'I_c': I_c,
        'demand_scenarios': demand_scenarios,
        'capacity_vector': capacity_vector,
        'is_existing': is_existing,
        'station_is_commercial_by_nearest': station_is_commercial_by_nearest
    }

# -------------------- helper: build cost vector for alpha --------------------

def build_cost_vector(alpha: float, is_existing: np.ndarray, station_is_commercial_by_nearest: np.ndarray) -> np.ndarray:
    J = len(is_existing)
    cost_vector = np.zeros(J, dtype=float)
    for j in range(J):
        if is_existing[j] and station_is_commercial_by_nearest[j]:
            cost_vector[j] = alpha * COST_EXISTING
        elif is_existing[j]:
            cost_vector[j] = COST_EXISTING
        elif station_is_commercial_by_nearest[j]:
            cost_vector[j] = alpha * COST_NEW
        else:
            cost_vector[j] = COST_NEW
    return cost_vector

# -------------------- run sensitivity --------------------

def run_alpha_dominance_analysis(fixed_budget: float, alpha_grid: np.ndarray):
    # load data for one representative strategy fileset (station files differ by strategy)
    # We need to load stations per strategy because stations CSV path uses strategy name.
    # For fairness we'll load station files separately inside the loop when necessary.

    records = []

    for alpha in alpha_grid:
        row = {'alpha': float(alpha)}
        strategy_results = {}
        for s in strategy_list:
            # load data for this strategy (stations file differs)
            data = load_data_for_strategy(s)
            pois = data['pois']
            stations = data['stations']
            D_km = data['D_km']
            I_c = data['I_c']
            demand_scenarios = data['demand_scenarios']
            capacity_vector = data['capacity_vector']
            is_existing = data['is_existing']
            station_is_commercial_by_nearest = data['station_is_commercial_by_nearest']

            cost_vector = build_cost_vector(alpha, is_existing, station_is_commercial_by_nearest)

            sol = build_and_solve_model(
                D_km, stations, pois, demand_scenarios, capacity_vector,
                cost_vector, I_c, W_C, Tau, k, float(fixed_budget), verbose=False
            )

            if sol is None:
                strategy_results[s] = {'feasible': False, 'Q': np.nan, 'budget_used': np.nan, 'opened': np.nan}
            else:
                strategy_results[s] = {'feasible': True, 'Q': float(sol['Q']), 'budget_used': float(sol['budget_used']), 'opened': int(sol['opened'])}

            # add to row
            row[f'{s}_feasible'] = strategy_results[s]['feasible']
            row[f'{s}_Q'] = strategy_results[s]['Q']
            row[f'{s}_budget_used'] = strategy_results[s]['budget_used']
            row[f'{s}_opened'] = strategy_results[s]['opened']

        # dominance check relative to uniform (Option C)
        # requires feasible for both
        u = strategy_results['uniform']
        for s in ['commercial', 'pred']:
            v = strategy_results[s]
            dominates = False
            if u['feasible'] and v['feasible']:
                leqQ = (v['Q'] <= u['Q'])
                leqB = (v['budget_used'] <= u['budget_used'])
                strict = (v['Q'] < u['Q']) or (v['budget_used'] < u['budget_used'])
                dominates = bool(leqQ and leqB and strict)
            row[f'{s}_dominates_uniform'] = dominates

        records.append(row)
        print(f'alpha={alpha:.3f}: commercial dominates uniform={row["commercial_dominates_uniform"]}, pred dominates uniform={row["pred_dominates_uniform"]}')

    df = pd.DataFrame(records)
    return df

# -------------------- plotting & reporting --------------------

def plot_results(df: pd.DataFrame, out_prefix: str):
    plt.figure(figsize=(10,6))
    for s in strategy_list:
        feasible_mask = df[f'{s}_feasible'] == True
        plt.plot(df.loc[feasible_mask, 'alpha'], df.loc[feasible_mask, f'{s}_Q'], marker='o', label=s)
    # mark alphas where dominance occurs
    comm_dom = df[df['commercial_dominates_uniform'] == True]['alpha']
    pred_dom = df[df['pred_dominates_uniform'] == True]['alpha']
    if not comm_dom.empty:
        plt.scatter(comm_dom, np.interp(comm_dom, df['alpha'], df['commercial_Q']), color='C1', s=120, marker='*', label='commercial dominates')
    if not pred_dom.empty:
        plt.scatter(pred_dom, np.interp(pred_dom, df['alpha'], df['pred_Q']), color='C2', s=120, marker='P', label='pred dominates')

    plt.xlabel('alpha')
    plt.ylabel('Optimal Q (km)')
    plt.title(f'Alpha sensitivity and dominance vs uniform (budget={fixed_budget:,.0f})')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    out_png = os.path.join(OUTPUT_DIR, f'{out_prefix}_alpha_vs_Q.png')
    plt.savefig(out_png, dpi=200)
    print(f'Saved plot to {out_png}')

    # Save CSV
    out_csv = os.path.join(OUTPUT_DIR, f'{out_prefix}_alpha_dominance.csv')
    df.to_csv(out_csv, index=False)
    print(f'Saved CSV to {out_csv}')

    # Report alpha thresholds: first alpha where dominance is True (if any)
    comm_first = df[df['commercial_dominates_uniform'] == True]['alpha']
    pred_first = df[df['pred_dominates_uniform'] == True]['alpha']
    print('Thresholds:')
    print('commercial first dominates at alpha=', float(comm_first.iloc[0]) if not comm_first.empty else 'never in grid')
    print('pred first dominates at alpha=', float(pred_first.iloc[0]) if not pred_first.empty else 'never in grid')

In [3]:
import folium
from folium import Circle, PolyLine

# Step 1: Load data
data = load_data_for_strategy('uniform')
pois = data['pois']
stations = data['stations']
D_km = data['D_km']
I_c = data['I_c']
demand_scenarios = data['demand_scenarios']
capacity_vector = data['capacity_vector']
is_existing = data['is_existing']
station_is_commercial_by_nearest = data['station_is_commercial_by_nearest']

# Step 2: Compute cost vector as in your pipeline
alpha = ALPHA
cost_vector = np.zeros(stations.shape[0], dtype=float)
for j in range(stations.shape[0]):
    if is_existing[j] and station_is_commercial_by_nearest[j]:
        cost_vector[j] = alpha * COST_EXISTING
    elif is_existing[j]:
        cost_vector[j] = COST_EXISTING
    elif station_is_commercial_by_nearest[j]:
        cost_vector[j] = alpha * COST_NEW
    else:
        cost_vector[j] = COST_NEW

# Step 3: Solve model at target budget
sol = build_and_solve_model(D_km, stations, pois, demand_scenarios,
                            capacity_vector, cost_vector, I_c, W_C,
                            Tau, k, fixed_budget, verbose=False)

if sol is None:
    print(f"No feasible solution found for budget £{fixed_budget}")
else:
    # Extract variables for mapping
    X_sol = sol['X_sol']
    I, J = D_km.shape

    # Build assignment matrix
    solution_matrix = np.zeros((I, J))
    for (i, j), val in X_sol.items():
        solution_matrix[i, j] = val

    # Assign each POI to station with max X_ij
    assignments = solution_matrix.argmax(axis=1)

    stations_open = sol['opened_indices']

    # Step 4: Plot map function
    def plot_station_poi_map(pois, stations, distance_matrix, station_indices_open, assignments, map_filename='stations_with_lines_map.html'):
        map_center = (pois['lat'].mean(), pois['lon'].mean())
        m = folium.Map(location=map_center, zoom_start=13)

        for station_idx in station_indices_open:
            station_lat = stations.iloc[station_idx]['centroid_lat']
            station_lon = stations.iloc[station_idx]['centroid_lon']

            # POIs served by this station
            served_pois = [i for i, j in enumerate(assignments) if j == station_idx]

            max_distance_km = max((distance_matrix[i, station_idx] for i in served_pois), default=0.1)

            folium.Marker(
                location=(station_lat, station_lon),
                popup=f"Station {stations.iloc[station_idx]['candidate_id']}",
                icon=folium.Icon(color='darkgreen', icon='bicycle', prefix='fa')
            ).add_to(m)

            Circle(
                location=(station_lat, station_lon),
                radius=max_distance_km * 1000,
                color='blue',
                fill=True,
                fill_opacity=0.1,
                weight=2,
                popup=f"Coverage area: {max_distance_km:.2f} km"
            ).add_to(m)

            for poi_idx in served_pois:
                poi_lat = pois.iloc[poi_idx]['lat']
                poi_lon = pois.iloc[poi_idx]['lon']

                folium.CircleMarker(
                    location=(poi_lat, poi_lon),
                    radius=5,
                    weight=3,
                    color='brown',
                    fill=True,
                    fill_opacity=1,
                    popup=f"POI {pois.iloc[poi_idx]['poi_id']}"
                ).add_to(m)

                PolyLine([(station_lat, station_lon), (poi_lat, poi_lon)],
                         color='black', weight=3, opacity=0.7
                ).add_to(m)

        m.save(map_filename)
        print(f"Map saved as {map_filename}")

    # Generate map
    plot_station_poi_map(pois, stations, D_km, stations_open, assignments, 'uniform_strategy_map_budget_8105409.html')


/var/folders/tn/v19h6ld131n5p0bkj6b850sr0000gn/T/ipykernel_82505/331104810.py:79: LicenseWarning: Using the license file found in your Xpress installation. If you want to use this license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('/Applications/FICO Xpress/xpressmp/bin/xpauth.xpr')
  prob = xp.problem(name='pcentre_modular_run')


Map saved as uniform_strategy_map_budget_8105409.html


In [10]:
stations_open = []
Y_sol=sol['Y_sol']
for j in range(J):
    if Y_sol[j] > 0.5:  # Binary variable, so > 0.5 means 1
        stations_open.append(j)

In [11]:
print(f"\n1. STATIONS TO OPEN ({len(stations_open)} out of {J}):")


1. STATIONS TO OPEN (83 out of 383):


In [18]:
sol['budget_used']

6750000.0

In [19]:
sol['Q']

1.1991829890904042

In [14]:
I_c

{'library': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43],
 'residential': [240,
  241,
  242,
  243,
  244,
  245,
  246,
  247,
  248,
  249,
  250,
  251,
  252,
  253,
  254,
  255,
  256,
  257,
  258,
  259,
  260,
  261,
  262,
  263,
  264,
  265,
  266,
  267,
  268,
  269,
  270,
  271,
  272,
  273,
  274,
  275,
  276,
  277,
  278,
  279,
  280,
  281,
  282,
  283,
  284,
  285,
  286,
  287,
  288,
  289,
  290,
  291,
  292,
  293,
  294,
  295,
  296,
  297,
  298,
  299,
  300,
  301,
  302,
  303,
  304,
  305,
  306,
  307,
  308,
  309,
  310,
  311,
  312,
  313,
  314,
  315,
  316,
  317,
  318,
  319,
  320,
  321,
  322,
  323,
  324,
  325,
  326,
  327,
  328,
  329,
  330,
  331,
  332,
  333,
  334,
  335,
  336,
  337,
  338,
  339,
  340,
  341,
  342

In [15]:
len(iI_c)

NameError: name 'iI_c' is not defined

In [ ]:
for c, idx_list in I_c.items():
    print(len(idx_list))


44
459
173
172
23
14
